Test récupération data

In [32]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import re

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# ... previous code ...


def fetch_episode_data(url):
    # Récupération de la page web
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # s'assure que la requête a réussi
    soup = BeautifulSoup(response.content, 'html.parser')

    # Récupérer le mois courant pour filtrer les épisodes
    current_month = datetime.now().strftime("%B").lower()

    data = []

    # Find all the 'td' elements with the class 'floatleftmobile td_jour'
    days = soup.find_all('td', class_='floatleftmobile td_jour')

    for day in days:
        class_regex = re.compile("div_jour(courant)?")
        # Find the date within the 'div' with class 'div_jour'
        date_div = day.find('div', class_=class_regex)
        if date_div:
            date_text = date_div.get('id')

            # Find all the series entries within the 'span' with class 'calendrier_episodes'
            episodes = day.find_all('span', class_='calendrier_episodes')
            for episode in episodes:
                # Extracting information using the tags and structure provided
                series_info = episode.find('a', style=True)
                episode_info = episode.find('a', class_='liens')
                country_img = episode.find_previous_sibling('img')
                network_img = country_img.find_next_sibling('img')

                name = series_info.get('title')
                episode_detail = episode_info.get('alt')
                network = country_img.get(
                    'alt') if country_img else None
                origin_country = network_img.get('alt') if network_img else None
                episode_url = episode_info.get('href')

                # Parse out the season and episode numbers from the text
                season_episode_match = re.search(
                    r'saison (\d+) episode (\d+)', episode_detail)
                if season_episode_match:
                    season_num = int(season_episode_match.group(1))
                    episode_num = int(season_episode_match.group(2))

                    # Add to the data list
                    episode_data = {
                        'name': name,
                        'season_num': season_num,
                        'episode_num': episode_num,
                        # Format the date as required
                        'date': date_text,
                        'origin_country': origin_country,
                        'network': network,
                        'episode_url': episode_url
                    }
                    data.append(episode_data)

    return data


# URL du site
url = 'https://www.spin-off.fr/calendrier_des_series.html'
episode_data = fetch_episode_data(url)

# Afficher les données récupérées
for episode in episode_data:
    print(episode)


NameError: name 'headers' is not defined

Mettre tous les éléments dans un fichier .CSV

In [33]:
import csv
import os

# Chemin vers le dossier où le fichier CSV sera enregistré
folder_path = 'data/files'
file_name = 'episodes.csv'
file_path = os.path.join(folder_path, file_name)
episode_data = fetch_episode_data(url)

# Créer le dossier s'il n'existe pas déjà
os.makedirs(folder_path, exist_ok=True)

# Enregistrement dans le fichier CSV
with open(file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file, delimiter=';')

    # Écriture de l'en-tête du fichier CSV
    writer.writerow(['nom_serie', 'numero_de_lepisode', 'numero_de_la_saison', 'date', 'pays_origine', 'reseau', 'url_episode'])

    # Écriture des données des épisodes
    for episode in episode_data:
        writer.writerow([
            episode['name'],
            episode['episode_num'],
            episode['season_num'],
            episode['date'],
            episode['origin_country'],
            episode['network'],
            episode['episode_url']
        ])

print(f"Les données ont été enregistrées dans '{file_path}'")


Les données ont été enregistrées dans 'data/files\episodes.csv'


sqllite add to table



In [34]:
import sqlite3
import os

# Supposons que episodes_data est votre liste de dictionnaires contenant toutes les données d'épisode
episode_data = fetch_episode_data(url)

# Chemin vers le dossier où la base de données SQLite sera enregistrée
db_folder_path = 'data/databases'
# Créer le dossier s'il n'existe pas déjà
os.makedirs(db_folder_path, exist_ok=True)

# Chemin complet de la base de données SQLite
db_file_path = os.path.join(db_folder_path, 'database.db')

# Connection à la base de données SQLite
conn = sqlite3.connect(db_file_path)
cursor = conn.cursor()

# Créer la table `episode`
cursor.execute('''
CREATE TABLE IF NOT EXISTS episode (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    season_num INTEGER NOT NULL,
    episode_num INTEGER NOT NULL,
    date TEXT,
    origin_country TEXT,
    network TEXT,
    episode_url TEXT
)
''')

# Insérer les données dans la table `episode`
for episode in episode_data:
    cursor.execute('''
    INSERT INTO episode (name, season_num, episode_num, date, origin_country, network, episode_url)
    VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', (
        episode['name'],
        episode['season_num'],
        episode['episode_num'],
        episode['date'],
        episode['origin_country'],
        episode['network'],
        episode['episode_url']
    ))

# Valider les changements
conn.commit()

# Fermer la connexion
conn.close()

print(f"Les données ont été insérées dans la base de données '{db_file_path}'")


Les données ont été insérées dans la base de données 'data/databases\database.db'


Question algorithmique

In [35]:
import sqlite3
import pandas as pd

# Connect to SQLite database
conn = sqlite3.connect('data/databases/database.db')

# SQL query to count episodes per network
query = """
SELECT network, COUNT(*) as episode_count
FROM episode
GROUP BY network
ORDER BY episode_count DESC
LIMIT 3;
"""

# Execute the SQL query and fetch the results
df = pd.read_sql_query(query, conn)

# Close the connection
conn.close()

# Print the results
print(df)

# Write the results to README.md
with open('README.md', 'a') as f:
    f.write("\n## Top 3 Networks with Most Episodes Broadcasted\n")
    for index, row in df.iterrows():
        f.write(f"- {row['network']}: {row['episode_count']} episodes\n")


     network  episode_count
0    Netflix             78
1    Disney+             64
2  Apple TV+             34


Faire de même pour les pays


In [36]:
import sqlite3

def write_top_counts_to_readme(category, limit=3):
    # Connect to SQLite database
    with sqlite3.connect('data/databases/database.db') as conn:
        # Create a cursor object using the cursor() method
        cursor = conn.cursor()
        
        # SQL query to count episodes per category (network or country)
        query = f"""
        SELECT {category}, COUNT(*) as episode_count
        FROM episode
        GROUP BY {category}
        ORDER BY episode_count DESC
        LIMIT {limit};
        """
        
        # Execute the SQL query
        cursor.execute(query)

        # Fetch all results
        results = cursor.fetchall()

        # Header for the README section
        readme_header = f"\n## Top {limit} Countries with Most Episodes Broadcasted\n" if category == 'origin_country' \
            else f"\n## Top {limit} Networks with Most Episodes Broadcasted\n"

        # Write the results to README.md
        with open('README.md', 'a') as f:
            f.write(readme_header)
            for row in results:
                f.write(f"- {row[0]}: {row[1]} episodes\n")

# Write top networks to README.md
write_top_counts_to_readme('network')

# Write top countries to README.md
write_top_counts_to_readme('origin_country')



3️⃣ Écrire une fonction ou une classe qui permet de lire le fichier episodes.csv sans utiliser de librairie. Cette fonction ou classe devra renvoyer une liste de tuples avec les bons types : 

In [12]:
# Définissez le chemin vers votre fichier CSV
csv_file_path = 'data/files/episodes.csv'

def read_csv_file_no_csv_module(file_path):
    episodes = []
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        for line in lines[1:]:  # Skip header line
            if line.strip():  # Skip empty lines
                # Split the line into fields
                fields = line.strip().split(';')
                # Cast each field to the correct type
                try:
                    name = str(fields[0])
                    episode_num = int(fields[1])
                    season_num = int(fields[2])
                    date = str(fields[3])
                    origin_country = str(fields[4])
                    network = str(fields[5])
                    episode_url = str(fields[6])
                    episode_tuple = (name, episode_num, season_num, date, origin_country, network, episode_url)
                    episodes.append(episode_tuple)
                except ValueError as e:
                    print(f"Error converting types for row {fields}: {e}")
    return episodes

# Use this function if you cannot use the csv module
episode_list = read_csv_file_no_csv_module(csv_file_path)

print(episode_list)


[('4 Estrellas', 110, 1, 'jour_01-11-2023', 'Etats-Unis', 'TVE', 'episode110-411772-01112023-saison1-4-Estrellas.html'), ('Black Cake', 1, 1, 'jour_01-11-2023', 'Etats-Unis', 'Hulu', 'episode01-410147-01112023-saison1-Black-Cake.html'), ('Black Cake', 2, 1, 'jour_01-11-2023', 'Etats-Unis', 'Hulu', 'episode02-410148-01112023-saison1-Black-Cake.html'), ('Black Cake', 3, 1, 'jour_01-11-2023', 'Italie', 'Hulu', 'episode03-410149-01112023-saison1-Black-Cake.html'), ('Blanca', 5, 2, 'jour_01-11-2023', 'Canada', 'Rai 1', 'episode05-411693-01112023-saison2-Blanca.html'), ('Children Ruin Everything', 6, 3, 'jour_01-11-2023', 'Etats-Unis', 'CTV', 'episode06-411350-01112023-saison3-Children-Ruin-Everything.html'), ('Chucky', 5, 3, 'jour_01-11-2023', 'Etats-Unis', 'Syfy', 'episode05-409118-01112023-saison3-Chucky.html'), ("Cooper's Bar", 5, 2, 'jour_01-11-2023', 'France', 'AMC', 'episode05-411065-01112023-saison2-Cooper-s-Bar.html'), ('Demain nous appartient', 53, 7, 'jour_01-11-2023', 'Allemagne'

In [ ]:
� Quels mots reviennent le plus souvent dans les noms des séries ? (attention à ne compter qu’une seule fois chaque
série, et pas une fois chaque épisode)
Les indiquer dans le fichier README.md

In [13]:
import re
from collections import Counter
import sqlite3

# Extraction des noms uniques des séries à partir de la base de données SQLite
def extraire_noms_series_unicques():
    conn = sqlite3.connect('data/databases/database.db')
    cursor = conn.cursor()
    cursor.execute("SELECT DISTINCT name FROM episode")
    series_noms = set(name[0] for name in cursor.fetchall())
    conn.close()
    return series_noms

# Compter la fréquence des mots dans les noms des séries
def compter_mots(series_noms):
    comptage_mots = Counter()
    for nom in series_noms:
        mots = re.findall(r'\b\w+\b', nom.lower())
        comptage_mots.update(mots)
    return comptage_mots

# Écrire les mots les plus fréquents dans README.md
def ecrire_readme(mots_communs):
    with open('README.md', 'a', encoding='utf-8') as f:
        f.write("\n## Mots les plus fréquents dans les noms de séries\n")
        for mot, frequence in mots_communs:
            f.write(f"- {mot}: {frequence} occurrences\n")

# Exécution des fonctions
series_noms = extraire_noms_series_unicques()
comptage_mots = compter_mots(series_noms)
mots_communs = comptage_mots.most_common()
ecrire_readme(mots_communs)


Test pour réupérer l'url de la page recherche 

In [51]:
import requests
from bs4 import BeautifulSoup
import time
import sqlite3

# Connexion à la base de données SQLite pour récupérer les URLs et les IDs des épisodes
conn = sqlite3.connect('data/databases/database.db')
cur = conn.cursor()

# Sélectionnez les IDs et les URLs des épisodes du réseau "apple-tv"
cur.execute("SELECT id, episode_url FROM episode WHERE network = 'Apple TV+'")
episodes_to_scrape = cur.fetchall()
conn.close()  # Fermez la connexion à la base de données

# Dictionnaire pour stocker les durées des épisodes
episode_durations = {}

def scrape_episode_duration(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    duration_tag = soup.find(class_='episode_infos_episode_format')
    duration_text = duration_tag.get_text(strip=True) if duration_tag else "Durée non trouvée"
    return duration_text

# Bouclez sur les épisodes et scrapez les durées
for episode_id, episode_url in episodes_to_scrape:

    # Assurez-vous que l'URL est complète
    if not episode_url.startswith('http'):
        episode_url = f'https://www.spin-off.fr/{episode_url}'

    # Scrapez la durée de l'épisode
    try:
        duration = scrape_episode_duration(episode_url)
        episode_durations[episode_id] = duration
        time.sleep(1)  # Pause pour éviter le rate limiting
    except Exception as e:
        print(f"Erreur lors du scraping de l'épisode {episode_id} à l'URL {episode_url}: {e}")

# Ici, `episode_durations` contient les durées des épisodes avec les IDs correspondants.
print(episode_durations)


{25: '50minutes', 92: '52minutes', 93: '52minutes', 94: '52minutes', 95: '50minutes', 127: '45minutes', 130: '54minutes', 174: '52minutes', 207: '45minutes', 211: '54minutes', 212: 'minutes', 213: 'minutes', 242: '52minutes', 254: '45minutes', 256: '54minutes', 257: 'minutes', 277: '52minutes'}
